Message sent from the perspective of a human, using the `HumanMessage` interface

In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI()
prompt = [HumanMessage("What is the capital of France?")]

model.invoke(prompt)

AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLu8LhXO34UxlrshmHvIXI9BWQVdk', 'finish_reason': 'stop', 'logprobs': None}, id='run-13b39adc-247d-4ecb-a742-ecc44ac41c6d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Adding a System Message
Using the `SystemMessage` interface to set instructions that the AI should follow.

In [2]:
from langchain_core.messages import SystemMessage
system_msg = SystemMessage('''
You are a helpful assistant that responds to questions with a three exclamation marks.''')

human_msg = prompt[0]

model.invoke([system_msg, human_msg])

AIMessage(content='Paris!!!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 35, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLu8McVTavfoGNIMWZ7KAP1C02D1e', 'finish_reason': 'stop', 'logprobs': None}, id='run-9bf80c1a-93c2-4c45-bdc1-65fa4d915044-0', usage_metadata={'input_tokens': 35, 'output_tokens': 3, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Prompt templates
Using `ChatPromptTemplate` to make prompts reusable

In [3]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ('system', '''Answer the question based on the context below. If the question cannot be answered using the
        information provided, answer with "I do not know".'''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])
prompt = template.invoke({
    "context": 'a capital city is usually where the government of a region is located',
    'question': 'What is the capital city of Kenya?'
})

model.invoke(prompt)

AIMessage(content='The capital city of Kenya is Nairobi.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 70, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLu8MJDTP1ghWR9WmkIFEUAD2gOZJ', 'finish_reason': 'stop', 'logprobs': None}, id='run-7698b484-f98c-466b-a709-8a3f30d337d8-0', usage_metadata={'input_tokens': 70, 'output_tokens': 9, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## defining the output format
Output can be configured, common ones being json, csv etc

In [8]:
from pydantic import BaseModel

class AnswerWithJustification(BaseModel):
    answer: str
    '''The answer to the question'''
    justification: str
    '''Justification for the answer'''

llm = ChatOpenAI(model='gpt-3.5-turbo' , temperature=0)
structured_llm = llm.with_structured_output(AnswerWithJustification)
structured_llm.invoke('''What weighs more, a pound of bricks or a pound of feathers?''')

/home/unr3liable/.local/share/virtualenvs/learning-langchain-XCSQtIIB/lib/python3.13/site-packages/langchain_openai/chat_models/base.py:1637: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


AnswerWithJustification(answer='They weigh the same.', justification='Both a pound of bricks and a pound of feathers weigh one pound. The weight is the same, but the volume and density of the two substances differ.')

## Common interface
- invoke: transform a single nput into an output
- batch: transform multiple inputs, into multiple outputs
- stream: streams output as they become available

__Bonus__: you can decorate any function using `@chain` decorator

In [5]:
from langchain_core.runnables import chain

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])

model = ChatOpenAI()

@chain
def chatbot(values):
    prompt = template.invoke(values)
    return model.invoke(prompt)

chatbot.invoke({"question": "which model providers offer LLMs?"})

AIMessage(content='Model providers like Lionel LLC, MTH Electric Trains, and Kato USA offer an array of LLMs (Limited Liability Models) for train enthusiasts and collectors. These models are typically based on real-life trains and often come in limited edition releases.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 25, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLu8OWBf6VnWESNMB9GqiSwMV2lZN', 'finish_reason': 'stop', 'logprobs': None}, id='run-3a7c820f-70c3-4194-b172-c8fd2d19ac6d-0', usage_metadata={'input_tokens': 25, 'output_tokens': 52, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning

### example using streams
augment the chatbot function

In [6]:
@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in model.stream(prompt):
        yield token

for part in chatbot.stream({
    'question': "What is the capital of Kenya?"
}):
    print(part)

content='' additional_kwargs={} response_metadata={} id='run-0fe05ef6-b047-4ad1-9805-55b10b0dfa14'
content='The' additional_kwargs={} response_metadata={} id='run-0fe05ef6-b047-4ad1-9805-55b10b0dfa14'
content=' capital' additional_kwargs={} response_metadata={} id='run-0fe05ef6-b047-4ad1-9805-55b10b0dfa14'
content=' of' additional_kwargs={} response_metadata={} id='run-0fe05ef6-b047-4ad1-9805-55b10b0dfa14'
content=' Kenya' additional_kwargs={} response_metadata={} id='run-0fe05ef6-b047-4ad1-9805-55b10b0dfa14'
content=' is' additional_kwargs={} response_metadata={} id='run-0fe05ef6-b047-4ad1-9805-55b10b0dfa14'
content=' Nairobi' additional_kwargs={} response_metadata={} id='run-0fe05ef6-b047-4ad1-9805-55b10b0dfa14'
content='.' additional_kwargs={} response_metadata={} id='run-0fe05ef6-b047-4ad1-9805-55b10b0dfa14'
content='' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'} id='run-0fe05ef6-b047-4ad1-9805-55b10b0dfa14'


## LCEL (LangChain Expression Language)
declarative language that optimizes execution plan (similar to apache spark)

In [7]:
chatbot = template | model

for part in chatbot.stream({
    'question': 'What is the gestation period for an elephant?'
}):
    print(part)

content='' additional_kwargs={} response_metadata={} id='run-ece8a32d-ee45-4cc9-80cb-2557e96a3210'
content='The' additional_kwargs={} response_metadata={} id='run-ece8a32d-ee45-4cc9-80cb-2557e96a3210'
content=' gest' additional_kwargs={} response_metadata={} id='run-ece8a32d-ee45-4cc9-80cb-2557e96a3210'
content='ation' additional_kwargs={} response_metadata={} id='run-ece8a32d-ee45-4cc9-80cb-2557e96a3210'
content=' period' additional_kwargs={} response_metadata={} id='run-ece8a32d-ee45-4cc9-80cb-2557e96a3210'
content=' for' additional_kwargs={} response_metadata={} id='run-ece8a32d-ee45-4cc9-80cb-2557e96a3210'
content=' an' additional_kwargs={} response_metadata={} id='run-ece8a32d-ee45-4cc9-80cb-2557e96a3210'
content=' elephant' additional_kwargs={} response_metadata={} id='run-ece8a32d-ee45-4cc9-80cb-2557e96a3210'
content=' is' additional_kwargs={} response_metadata={} id='run-ece8a32d-ee45-4cc9-80cb-2557e96a3210'
content=' approximately' additional_kwargs={} response_metadata={} id=